<a id="item31"></a>


## Regression with Keras assignment: In this part we use the normalized data instead of the original raw data for training


In [4]:
# All Libraries required for this lab are listed below. The libraries pre-installed on Skills Network Labs are commented. 
# If you run this notebook on a different environment, e.g. your desktop, you may need to uncomment and install certain libraries.

#!pip install numpy==2.0.2
#!pip install pandas==2.2.2
#!pip install tensorflow_cpu==2.18.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 88.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.1
    Uninstalling numpy-2.2.1:
      Successfully uninstalled numpy-2.2.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.2/230.2 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 79.7 kB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 64.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 52.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 23.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━

## Importing the required libraries

In [1]:
import pandas as pd
import numpy as np
import keras

import warnings
warnings.simplefilter('ignore', FutureWarning)

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from sklearn.metrics import mean_squared_error

In [24]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

## Read the data

In [5]:
filepath='concrete_data.csv'
concrete_data = pd.read_csv(filepath)

concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [6]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [7]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

## Separating the predictors and the target

In [8]:
concrete_data_columns = concrete_data.columns

In [9]:
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

<a id="item2"></a>


## Normalizing the data

In [10]:
predictors_normalized = (predictors - predictors.mean()) / predictors.std()
predictors_normalized.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


## Train-test splitting of the data

In [13]:
X_train, X_test, y_train, y_test = train_test_split(predictors_normalized, target, test_size=0.30, random_state=42)

## Building a neural network with keras using 1 hidden layer of 10 nodes,  ReLu activation function, adam optimizer and the mean squared error  as the loss function.

In [14]:
def regression_model():
    # create model
    model = Sequential()
    model.add(Input(shape=(predictors.shape[1],))) # input layer with number of predictors
    model.add(Dense(10, activation='relu'))
    
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

## Model initialization

In [15]:
model = regression_model()

## Model training by fitting the model instance on the training data

In [16]:
model.fit(X_train, y_train, validation_split=0.3, epochs=50, verbose=2)

Epoch 1/50
16/16 - 9s - loss: 1576.8514 - val_loss: 1453.2886 - 9s/epoch - 538ms/step
Epoch 2/50
16/16 - 0s - loss: 1565.9700 - val_loss: 1443.2518 - 204ms/epoch - 13ms/step
Epoch 3/50
16/16 - 0s - loss: 1555.0278 - val_loss: 1432.7581 - 206ms/epoch - 13ms/step
Epoch 4/50
16/16 - 0s - loss: 1543.4423 - val_loss: 1421.9697 - 167ms/epoch - 10ms/step
Epoch 5/50
16/16 - 0s - loss: 1531.4397 - val_loss: 1410.7814 - 152ms/epoch - 10ms/step
Epoch 6/50
16/16 - 0s - loss: 1518.8342 - val_loss: 1399.1389 - 151ms/epoch - 9ms/step
Epoch 7/50
16/16 - 0s - loss: 1505.9224 - val_loss: 1386.6875 - 173ms/epoch - 11ms/step
Epoch 8/50
16/16 - 0s - loss: 1491.7512 - val_loss: 1374.1271 - 185ms/epoch - 12ms/step
Epoch 9/50
16/16 - 0s - loss: 1477.5569 - val_loss: 1360.5216 - 170ms/epoch - 11ms/step
Epoch 10/50
16/16 - 0s - loss: 1462.1027 - val_loss: 1346.2874 - 196ms/epoch - 12ms/step
Epoch 11/50
16/16 - 0s - loss: 1445.7618 - val_loss: 1331.4727 - 235ms/epoch - 15ms/step
Epoch 12/50
16/16 - 0s - loss: 14

## Calculating model predictions over the test data using model.predict()

In [17]:
predicted_strength = model.predict(X_test)

10/10 [==============================] - 0s 5ms/step


In [18]:
#predicted_strength

## Calculating mean squared error from mean_squared_error of scikit learn

In [19]:
mse = mean_squared_error(y_test, predicted_strength)

In [20]:
print('Mean squared error on the test data is: ', mse)

Mean squared error on the test data is:  464.2052093003634


## Repeating, splitting and model trainining 50 times and calculating the mse and standard deviation of MSE

In [21]:
mse_50 = []
model = regression_model()
for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.30)
    model.fit(X_train, y_train, validation_split=0.3, epochs=50, verbose=2)
    predictions = model.predict(X_test)
    mse_50.append(mean_squared_error(y_test, predictions))

Epoch 1/50
16/16 - 1s - loss: 100162.2188 - val_loss: 68771.9688 - 1s/epoch - 68ms/step
Epoch 2/50
16/16 - 0s - loss: 50872.4258 - val_loss: 31969.9121 - 212ms/epoch - 13ms/step
Epoch 3/50
16/16 - 0s - loss: 22651.2539 - val_loss: 13028.9697 - 206ms/epoch - 13ms/step
Epoch 4/50
16/16 - 0s - loss: 9063.3203 - val_loss: 5023.5503 - 194ms/epoch - 12ms/step
Epoch 5/50
16/16 - 0s - loss: 3657.3345 - val_loss: 2400.4485 - 196ms/epoch - 12ms/step
Epoch 6/50
16/16 - 0s - loss: 2064.7095 - val_loss: 1760.0339 - 261ms/epoch - 16ms/step
Epoch 7/50
16/16 - 0s - loss: 1682.4419 - val_loss: 1669.2549 - 240ms/epoch - 15ms/step
Epoch 8/50
16/16 - 0s - loss: 1623.2440 - val_loss: 1650.9170 - 253ms/epoch - 16ms/step
Epoch 9/50
16/16 - 0s - loss: 1590.1654 - val_loss: 1620.3264 - 238ms/epoch - 15ms/step
Epoch 10/50
16/16 - 0s - loss: 1560.1233 - val_loss: 1585.6157 - 217ms/epoch - 14ms/step
Epoch 11/50
16/16 - 0s - loss: 1529.1608 - val_loss: 1553.0623 - 209ms/epoch - 13ms/step
Epoch 12/50
16/16 - 0s - l

16/16 - 0s - loss: 261.9138 - val_loss: 261.3057 - 193ms/epoch - 12ms/step
Epoch 45/50
16/16 - 0s - loss: 257.2480 - val_loss: 255.5474 - 211ms/epoch - 13ms/step
Epoch 46/50
16/16 - 0s - loss: 253.8419 - val_loss: 252.4402 - 218ms/epoch - 14ms/step
Epoch 47/50
16/16 - 0s - loss: 249.0724 - val_loss: 245.8758 - 216ms/epoch - 14ms/step
Epoch 48/50
16/16 - 0s - loss: 243.9901 - val_loss: 244.4630 - 218ms/epoch - 14ms/step
Epoch 49/50
16/16 - 0s - loss: 240.8369 - val_loss: 238.0315 - 228ms/epoch - 14ms/step
Epoch 50/50
16/16 - 0s - loss: 236.5787 - val_loss: 234.4917 - 237ms/epoch - 15ms/step
10/10 [==============================] - 0s 4ms/step
Epoch 1/50
16/16 - 0s - loss: 226.8745 - val_loss: 243.3857 - 329ms/epoch - 21ms/step
Epoch 2/50
16/16 - 0s - loss: 222.8928 - val_loss: 240.1196 - 225ms/epoch - 14ms/step
Epoch 3/50
16/16 - 0s - loss: 219.1804 - val_loss: 235.9998 - 220ms/epoch - 14ms/step
Epoch 4/50
16/16 - 0s - loss: 216.4582 - val_loss: 232.7496 - 227ms/epoch - 14ms/step
Epoch 

Epoch 38/50
16/16 - 0s - loss: 115.9241 - val_loss: 106.9307 - 199ms/epoch - 12ms/step
Epoch 39/50
16/16 - 0s - loss: 116.3696 - val_loss: 106.0029 - 202ms/epoch - 13ms/step
Epoch 40/50
16/16 - 0s - loss: 115.0432 - val_loss: 105.7276 - 210ms/epoch - 13ms/step
Epoch 41/50
16/16 - 0s - loss: 114.9579 - val_loss: 105.1438 - 202ms/epoch - 13ms/step
Epoch 42/50
16/16 - 0s - loss: 115.0847 - val_loss: 106.1655 - 219ms/epoch - 14ms/step
Epoch 43/50
16/16 - 0s - loss: 114.8394 - val_loss: 104.5487 - 210ms/epoch - 13ms/step
Epoch 44/50
16/16 - 0s - loss: 114.0745 - val_loss: 104.9873 - 200ms/epoch - 12ms/step
Epoch 45/50
16/16 - 0s - loss: 114.4854 - val_loss: 105.2788 - 200ms/epoch - 12ms/step
Epoch 46/50
16/16 - 0s - loss: 113.9775 - val_loss: 104.2510 - 204ms/epoch - 13ms/step
Epoch 47/50
16/16 - 0s - loss: 113.6248 - val_loss: 103.7637 - 248ms/epoch - 16ms/step
Epoch 48/50
16/16 - 0s - loss: 113.6486 - val_loss: 103.2085 - 205ms/epoch - 13ms/step
Epoch 49/50
16/16 - 0s - loss: 113.2320 - v

Epoch 32/50
16/16 - 0s - loss: 107.9067 - val_loss: 115.1437 - 241ms/epoch - 15ms/step
Epoch 33/50
16/16 - 0s - loss: 110.7762 - val_loss: 118.8542 - 228ms/epoch - 14ms/step
Epoch 34/50
16/16 - 0s - loss: 107.6005 - val_loss: 113.0602 - 191ms/epoch - 12ms/step
Epoch 35/50
16/16 - 0s - loss: 105.5715 - val_loss: 114.2976 - 197ms/epoch - 12ms/step
Epoch 36/50
16/16 - 0s - loss: 106.1063 - val_loss: 117.9204 - 204ms/epoch - 13ms/step
Epoch 37/50
16/16 - 0s - loss: 105.9266 - val_loss: 109.9654 - 244ms/epoch - 15ms/step
Epoch 38/50
16/16 - 0s - loss: 104.8553 - val_loss: 109.9550 - 246ms/epoch - 15ms/step
Epoch 39/50
16/16 - 0s - loss: 104.9417 - val_loss: 116.2169 - 220ms/epoch - 14ms/step
Epoch 40/50
16/16 - 0s - loss: 107.4512 - val_loss: 114.6328 - 164ms/epoch - 10ms/step
Epoch 41/50
16/16 - 0s - loss: 107.7811 - val_loss: 111.7150 - 175ms/epoch - 11ms/step
Epoch 42/50
16/16 - 0s - loss: 106.5751 - val_loss: 109.8214 - 184ms/epoch - 12ms/step
Epoch 43/50
16/16 - 0s - loss: 105.2215 - v

Epoch 26/50
16/16 - 0s - loss: 101.7464 - val_loss: 150.2930 - 190ms/epoch - 12ms/step
Epoch 27/50
16/16 - 0s - loss: 101.7024 - val_loss: 144.2368 - 199ms/epoch - 12ms/step
Epoch 28/50
16/16 - 0s - loss: 107.8131 - val_loss: 152.5747 - 198ms/epoch - 12ms/step
Epoch 29/50
16/16 - 0s - loss: 103.5343 - val_loss: 140.2283 - 205ms/epoch - 13ms/step
Epoch 30/50
16/16 - 0s - loss: 101.5322 - val_loss: 138.9323 - 249ms/epoch - 16ms/step
Epoch 31/50
16/16 - 0s - loss: 102.1079 - val_loss: 138.5531 - 216ms/epoch - 13ms/step
Epoch 32/50
16/16 - 0s - loss: 104.7251 - val_loss: 138.6004 - 208ms/epoch - 13ms/step
Epoch 33/50
16/16 - 0s - loss: 106.0066 - val_loss: 139.0594 - 208ms/epoch - 13ms/step
Epoch 34/50
16/16 - 0s - loss: 101.7382 - val_loss: 139.0306 - 203ms/epoch - 13ms/step
Epoch 35/50
16/16 - 0s - loss: 102.4203 - val_loss: 146.7082 - 220ms/epoch - 14ms/step
Epoch 36/50
16/16 - 0s - loss: 102.8291 - val_loss: 138.8024 - 200ms/epoch - 13ms/step
Epoch 37/50
16/16 - 0s - loss: 101.6768 - v

...

Epoch 36/50
16/16 - 0s - loss: 44.6765 - val_loss: 51.4361 - 202ms/epoch - 13ms/step
Epoch 37/50
16/16 - 0s - loss: 44.7973 - val_loss: 50.8110 - 207ms/epoch - 13ms/step
Epoch 38/50
16/16 - 0s - loss: 45.8612 - val_loss: 51.8269 - 202ms/epoch - 13ms/step
Epoch 39/50
16/16 - 0s - loss: 47.6175 - val_loss: 56.0538 - 217ms/epoch - 14ms/step
Epoch 40/50
16/16 - 0s - loss: 50.2183 - val_loss: 51.0459 - 215ms/epoch - 13ms/step
Epoch 41/50
16/16 - 0s - loss: 45.4935 - val_loss: 50.2197 - 203ms/epoch - 13ms/step
Epoch 42/50
16/16 - 0s - loss: 45.1524 - val_loss: 51.5000 - 215ms/epoch - 13ms/step
Epoch 43/50
16/16 - 0s - loss: 45.3162 - val_loss: 50.6339 - 215ms/epoch - 13ms/step
Epoch 44/50
16/16 - 0s - loss: 46.0499 - val_loss: 54.7506 - 213ms/epoch - 13ms/step
Epoch 45/50
16/16 - 0s - loss: 44.6402 - val_loss: 50.5944 - 210ms/epoch - 13ms/step
Epoch 46/50
16/16 - 0s - loss: 44.2988 - val_loss: 50.3140 - 222ms/epoch - 14ms/step
Epoch 47/50
16/16 - 0s - loss: 44.9631 - val_loss: 50.9414 - 215m

Epoch 32/50
16/16 - 0s - loss: 43.5648 - val_loss: 49.8002 - 196ms/epoch - 12ms/step
Epoch 33/50
16/16 - 0s - loss: 46.5206 - val_loss: 49.9356 - 211ms/epoch - 13ms/step
Epoch 34/50
16/16 - 0s - loss: 48.6695 - val_loss: 64.5735 - 220ms/epoch - 14ms/step
Epoch 35/50
16/16 - 0s - loss: 49.0964 - val_loss: 57.0226 - 205ms/epoch - 13ms/step
Epoch 36/50
16/16 - 0s - loss: 48.6642 - val_loss: 49.6877 - 214ms/epoch - 13ms/step
Epoch 37/50
16/16 - 0s - loss: 43.8935 - val_loss: 49.3818 - 194ms/epoch - 12ms/step
Epoch 38/50
16/16 - 0s - loss: 43.9057 - val_loss: 50.7276 - 217ms/epoch - 14ms/step
Epoch 39/50
16/16 - 0s - loss: 45.1163 - val_loss: 50.8677 - 209ms/epoch - 13ms/step
Epoch 40/50
16/16 - 0s - loss: 43.0213 - val_loss: 49.8923 - 221ms/epoch - 14ms/step
Epoch 41/50
16/16 - 0s - loss: 45.2091 - val_loss: 53.7951 - 253ms/epoch - 16ms/step
Epoch 42/50
16/16 - 0s - loss: 43.2492 - val_loss: 49.4924 - 232ms/epoch - 15ms/step
Epoch 43/50
16/16 - 0s - loss: 43.3500 - val_loss: 49.8049 - 224m

Epoch 28/50
16/16 - 0s - loss: 47.1284 - val_loss: 35.9806 - 229ms/epoch - 14ms/step
Epoch 29/50
16/16 - 0s - loss: 46.6035 - val_loss: 37.3468 - 202ms/epoch - 13ms/step
Epoch 30/50
16/16 - 0s - loss: 46.5996 - val_loss: 40.4157 - 240ms/epoch - 15ms/step
Epoch 31/50
16/16 - 0s - loss: 45.5642 - val_loss: 37.1360 - 218ms/epoch - 14ms/step
Epoch 32/50
16/16 - 0s - loss: 46.0534 - val_loss: 35.5318 - 207ms/epoch - 13ms/step
Epoch 33/50
16/16 - 0s - loss: 48.1405 - val_loss: 35.5662 - 237ms/epoch - 15ms/step
Epoch 34/50
16/16 - 0s - loss: 45.2677 - val_loss: 37.3698 - 218ms/epoch - 14ms/step
Epoch 35/50
16/16 - 0s - loss: 45.5573 - val_loss: 40.4608 - 251ms/epoch - 16ms/step
Epoch 36/50
16/16 - 0s - loss: 48.1739 - val_loss: 39.6381 - 214ms/epoch - 13ms/step
Epoch 37/50
16/16 - 0s - loss: 48.2579 - val_loss: 37.4272 - 227ms/epoch - 14ms/step
Epoch 38/50
16/16 - 0s - loss: 46.8272 - val_loss: 36.7157 - 244ms/epoch - 15ms/step
Epoch 39/50
16/16 - 0s - loss: 46.6177 - val_loss: 35.9334 - 211m

Epoch 24/50
16/16 - 0s - loss: 47.2625 - val_loss: 60.7282 - 214ms/epoch - 13ms/step
Epoch 25/50
16/16 - 0s - loss: 48.1605 - val_loss: 51.7318 - 251ms/epoch - 16ms/step
Epoch 26/50
16/16 - 0s - loss: 47.4584 - val_loss: 51.8912 - 221ms/epoch - 14ms/step
Epoch 27/50
16/16 - 0s - loss: 46.2311 - val_loss: 51.8907 - 198ms/epoch - 12ms/step
Epoch 28/50
16/16 - 0s - loss: 46.5570 - val_loss: 52.2293 - 225ms/epoch - 14ms/step
Epoch 29/50
16/16 - 0s - loss: 46.5084 - val_loss: 52.5902 - 234ms/epoch - 15ms/step
Epoch 30/50
16/16 - 0s - loss: 47.3881 - val_loss: 54.1345 - 216ms/epoch - 13ms/step
Epoch 31/50
16/16 - 0s - loss: 47.4847 - val_loss: 54.4366 - 206ms/epoch - 13ms/step
Epoch 32/50
16/16 - 0s - loss: 48.2561 - val_loss: 52.7421 - 213ms/epoch - 13ms/step
Epoch 33/50
16/16 - 0s - loss: 47.0880 - val_loss: 51.8100 - 199ms/epoch - 12ms/step
Epoch 34/50
16/16 - 0s - loss: 45.7978 - val_loss: 52.1749 - 216ms/epoch - 14ms/step
Epoch 35/50
16/16 - 0s - loss: 46.2029 - val_loss: 52.5365 - 219m

Epoch 20/50
16/16 - 0s - loss: 41.5997 - val_loss: 45.7699 - 207ms/epoch - 13ms/step
Epoch 21/50
16/16 - 0s - loss: 46.7767 - val_loss: 46.5186 - 209ms/epoch - 13ms/step
Epoch 22/50
16/16 - 0s - loss: 41.5665 - val_loss: 46.3090 - 213ms/epoch - 13ms/step
Epoch 23/50
16/16 - 0s - loss: 43.0021 - val_loss: 46.2107 - 204ms/epoch - 13ms/step
Epoch 24/50
16/16 - 0s - loss: 43.1246 - val_loss: 45.4993 - 207ms/epoch - 13ms/step
Epoch 25/50
16/16 - 0s - loss: 41.6680 - val_loss: 45.2736 - 212ms/epoch - 13ms/step
Epoch 26/50
16/16 - 0s - loss: 41.6414 - val_loss: 45.8883 - 217ms/epoch - 14ms/step
Epoch 27/50
16/16 - 0s - loss: 41.5916 - val_loss: 45.8226 - 221ms/epoch - 14ms/step
Epoch 28/50
16/16 - 0s - loss: 43.7763 - val_loss: 50.2397 - 209ms/epoch - 13ms/step
Epoch 29/50
16/16 - 0s - loss: 45.4375 - val_loss: 48.8926 - 222ms/epoch - 14ms/step
Epoch 30/50
16/16 - 0s - loss: 42.3167 - val_loss: 49.4687 - 203ms/epoch - 13ms/step
Epoch 31/50
16/16 - 0s - loss: 42.7148 - val_loss: 47.3672 - 218m

## Calculating mean and standard deviation of the MSEs using numpy 

In [22]:
mse_arr = np.array(mse_50)
mean_mse = np.mean(mse_arr)
std_mse = np.std(mse_arr)

In [23]:
print('The mean MSE after training the model 50 times is: ',mean_mse, ' and the standard deviation is: ', std_mse)

The mean MSE after training the model 50 times is:  92.61864332056368  and the standard deviation is:  85.06794677610809


In part A the mean MSE and standard deviation was 58.269382589504986 and 18.803301390418138 respectively. 
Training the model on a normalized data in this particular dataset has led to increase in the error and the resulting error and standard deviation in MSE is 92.61864332056368  and 85.06794677610809 respectively

## using the MinMax scaler to scale the data instead of subtracting the mean and dividing by the standard deviation

In [25]:
# Splitting the raw data into training and testing and using the minmax scaler instance to fit and transform the training data
#and using the same transform on the test data:

mse_50_Minmax = []
minmax = MinMaxScaler()
model = regression_model()
for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.30)
    X_train_scaled = minmax.fit_transform(X_train)
    X_test_scaled = minmax.transform(X_test)
    model.fit(X_train_scaled, y_train, validation_split=0.3, epochs=50, verbose=2)
    predictions = model.predict(X_test_scaled)
    mse_50_Minmax.append(mean_squared_error(y_test, predictions))






Epoch 1/50
16/16 - 1s - loss: 1634.5219 - val_loss: 1482.4193 - 1s/epoch - 94ms/step
Epoch 2/50
16/16 - 0s - loss: 1622.8274 - val_loss: 1470.9933 - 152ms/epoch - 9ms/step
Epoch 3/50
16/16 - 0s - loss: 1610.4081 - val_loss: 1458.5173 - 167ms/epoch - 10ms/step
Epoch 4/50
16/16 - 0s - loss: 1596.8229 - val_loss: 1444.5411 - 173ms/epoch - 11ms/step
Epoch 5/50
16/16 - 0s - loss: 1581.3524 - val_loss: 1428.1656 - 171ms/epoch - 11ms/step
Epoch 6/50
16/16 - 0s - loss: 1562.6427 - val_loss: 1408.6860 - 151ms/epoch - 9ms/step
Epoch 7/50
16/16 - 0s - loss: 1540.6851 - val_loss: 1386.1021 - 145ms/epoch - 9ms/step
Epoch 8/50
16/16 - 0s - loss: 1516.1047 - val_loss: 1361.8228 - 128ms/epoch - 8ms/step
Epoch 9/50
16/16 - 0s - loss: 1489.8262 - val_loss: 1336.4254 - 136ms/epoch - 9ms/step
Epoch 10/50
16/16 - 0s - loss: 1462.3156 - val_loss: 1309.1345 - 157ms/epoch - 10ms/step
Epoch 11/50
16/16 - 0s - loss: 1433.1158 - val_loss: 1280.8318 - 195ms/epoch - 12ms/step
Epoch 12/50
16/16 - 0s - loss: 1402.48

Epoch 45/50
16/16 - 0s - loss: 224.5410 - val_loss: 210.8817 - 259ms/epoch - 16ms/step
Epoch 46/50
16/16 - 0s - loss: 223.7206 - val_loss: 210.1602 - 254ms/epoch - 16ms/step
Epoch 47/50
16/16 - 0s - loss: 222.8846 - val_loss: 209.4972 - 297ms/epoch - 19ms/step
Epoch 48/50
16/16 - 0s - loss: 222.0203 - val_loss: 208.8576 - 236ms/epoch - 15ms/step
Epoch 49/50
16/16 - 0s - loss: 221.1788 - val_loss: 207.9848 - 231ms/epoch - 14ms/step
Epoch 50/50
16/16 - 0s - loss: 220.3149 - val_loss: 207.2804 - 229ms/epoch - 14ms/step
10/10 [==============================] - 0s 4ms/step
Epoch 1/50
16/16 - 0s - loss: 200.7360 - val_loss: 249.3483 - 319ms/epoch - 20ms/step
Epoch 2/50
16/16 - 0s - loss: 199.8020 - val_loss: 248.9186 - 223ms/epoch - 14ms/step
Epoch 3/50
16/16 - 0s - loss: 198.6986 - val_loss: 248.2547 - 262ms/epoch - 16ms/step
Epoch 4/50
16/16 - 0s - loss: 197.7593 - val_loss: 247.7529 - 254ms/epoch - 16ms/step
Epoch 5/50
16/16 - 0s - loss: 196.7939 - val_loss: 247.1456 - 238ms/epoch - 15ms/

Epoch 39/50
16/16 - 0s - loss: 145.1221 - val_loss: 141.4131 - 199ms/epoch - 12ms/step
Epoch 40/50
16/16 - 0s - loss: 144.3670 - val_loss: 140.8482 - 195ms/epoch - 12ms/step
Epoch 41/50
16/16 - 0s - loss: 143.6322 - val_loss: 140.2848 - 177ms/epoch - 11ms/step
Epoch 42/50
16/16 - 0s - loss: 142.8885 - val_loss: 139.7439 - 200ms/epoch - 13ms/step
Epoch 43/50
16/16 - 0s - loss: 142.1926 - val_loss: 139.1767 - 233ms/epoch - 15ms/step
Epoch 44/50
16/16 - 0s - loss: 141.4954 - val_loss: 138.5971 - 246ms/epoch - 15ms/step
Epoch 45/50
16/16 - 0s - loss: 140.7646 - val_loss: 138.0604 - 230ms/epoch - 14ms/step
Epoch 46/50
16/16 - 0s - loss: 140.0488 - val_loss: 137.5679 - 209ms/epoch - 13ms/step
Epoch 47/50
16/16 - 0s - loss: 139.4311 - val_loss: 137.0069 - 220ms/epoch - 14ms/step
Epoch 48/50
16/16 - 0s - loss: 138.7744 - val_loss: 136.5235 - 254ms/epoch - 16ms/step
Epoch 49/50
16/16 - 0s - loss: 138.0562 - val_loss: 135.9951 - 211ms/epoch - 13ms/step
Epoch 50/50
16/16 - 0s - loss: 137.4102 - v

Epoch 33/50
16/16 - 0s - loss: 110.1402 - val_loss: 110.6626 - 260ms/epoch - 16ms/step
Epoch 34/50
16/16 - 0s - loss: 109.9599 - val_loss: 110.2945 - 238ms/epoch - 15ms/step
Epoch 35/50
16/16 - 0s - loss: 109.7777 - val_loss: 110.2929 - 232ms/epoch - 15ms/step
Epoch 36/50
16/16 - 0s - loss: 109.6102 - val_loss: 110.3621 - 203ms/epoch - 13ms/step
Epoch 37/50
16/16 - 0s - loss: 109.4036 - val_loss: 110.1880 - 203ms/epoch - 13ms/step
Epoch 38/50
16/16 - 0s - loss: 109.1978 - val_loss: 109.8687 - 206ms/epoch - 13ms/step
Epoch 39/50
16/16 - 0s - loss: 109.0479 - val_loss: 109.5881 - 243ms/epoch - 15ms/step
Epoch 40/50
16/16 - 0s - loss: 108.8591 - val_loss: 109.6032 - 235ms/epoch - 15ms/step
Epoch 41/50
16/16 - 0s - loss: 108.6903 - val_loss: 109.7286 - 221ms/epoch - 14ms/step
Epoch 42/50
16/16 - 0s - loss: 108.5197 - val_loss: 109.4421 - 232ms/epoch - 15ms/step
Epoch 43/50
16/16 - 0s - loss: 108.3416 - val_loss: 109.3366 - 211ms/epoch - 13ms/step
Epoch 44/50
16/16 - 0s - loss: 108.2011 - v

Epoch 27/50
16/16 - 0s - loss: 107.9673 - val_loss: 93.0356 - 216ms/epoch - 13ms/step
Epoch 28/50
16/16 - 0s - loss: 107.8399 - val_loss: 92.9765 - 220ms/epoch - 14ms/step
Epoch 29/50
16/16 - 0s - loss: 107.7398 - val_loss: 93.0065 - 201ms/epoch - 13ms/step
Epoch 30/50
16/16 - 0s - loss: 107.6301 - val_loss: 92.9592 - 213ms/epoch - 13ms/step
Epoch 31/50
16/16 - 0s - loss: 107.5405 - val_loss: 92.9272 - 228ms/epoch - 14ms/step
Epoch 32/50
16/16 - 0s - loss: 107.4233 - val_loss: 92.9573 - 235ms/epoch - 15ms/step
Epoch 33/50
16/16 - 0s - loss: 107.3352 - val_loss: 92.9750 - 208ms/epoch - 13ms/step
Epoch 34/50
16/16 - 0s - loss: 107.3661 - val_loss: 92.8209 - 228ms/epoch - 14ms/step
Epoch 35/50
16/16 - 0s - loss: 107.0835 - val_loss: 92.8652 - 224ms/epoch - 14ms/step
Epoch 36/50
16/16 - 0s - loss: 106.9761 - val_loss: 92.8834 - 193ms/epoch - 12ms/step
Epoch 37/50
16/16 - 0s - loss: 106.9546 - val_loss: 92.9812 - 197ms/epoch - 12ms/step
Epoch 38/50
16/16 - 0s - loss: 106.7930 - val_loss: 92

...

Epoch 4/50
16/16 - 0s - loss: 46.0067 - val_loss: 43.7176 - 214ms/epoch - 13ms/step
Epoch 5/50
16/16 - 0s - loss: 46.0060 - val_loss: 43.7391 - 227ms/epoch - 14ms/step
Epoch 6/50
16/16 - 0s - loss: 45.9828 - val_loss: 43.7301 - 212ms/epoch - 13ms/step
Epoch 7/50
16/16 - 0s - loss: 45.9400 - val_loss: 43.7575 - 209ms/epoch - 13ms/step
Epoch 8/50
16/16 - 0s - loss: 45.9702 - val_loss: 43.7088 - 231ms/epoch - 14ms/step
Epoch 9/50
16/16 - 0s - loss: 45.9256 - val_loss: 43.7230 - 221ms/epoch - 14ms/step
Epoch 10/50
16/16 - 0s - loss: 45.9516 - val_loss: 43.7496 - 233ms/epoch - 15ms/step
Epoch 11/50
16/16 - 0s - loss: 45.8845 - val_loss: 43.6993 - 229ms/epoch - 14ms/step
Epoch 12/50
16/16 - 0s - loss: 45.8980 - val_loss: 43.6870 - 230ms/epoch - 14ms/step
Epoch 13/50
16/16 - 0s - loss: 45.8875 - val_loss: 43.6435 - 208ms/epoch - 13ms/step
Epoch 14/50
16/16 - 0s - loss: 45.9179 - val_loss: 43.6831 - 224ms/epoch - 14ms/step
Epoch 15/50
16/16 - 0s - loss: 45.9081 - val_loss: 43.6504 - 240ms/epoc

Epoch 50/50
16/16 - 0s - loss: 43.2226 - val_loss: 48.4411 - 234ms/epoch - 15ms/step
10/10 [==============================] - 0s 5ms/step
Epoch 1/50
16/16 - 0s - loss: 47.6819 - val_loss: 41.8272 - 316ms/epoch - 20ms/step
Epoch 2/50
16/16 - 0s - loss: 47.5672 - val_loss: 41.7383 - 242ms/epoch - 15ms/step
Epoch 3/50
16/16 - 0s - loss: 47.4807 - val_loss: 41.6852 - 240ms/epoch - 15ms/step
Epoch 4/50
16/16 - 0s - loss: 47.4093 - val_loss: 41.7144 - 224ms/epoch - 14ms/step
Epoch 5/50
16/16 - 0s - loss: 47.3417 - val_loss: 41.6022 - 236ms/epoch - 15ms/step
Epoch 6/50
16/16 - 0s - loss: 47.1165 - val_loss: 41.5676 - 221ms/epoch - 14ms/step
Epoch 7/50
16/16 - 0s - loss: 47.1079 - val_loss: 41.5592 - 232ms/epoch - 14ms/step
Epoch 8/50
16/16 - 0s - loss: 47.1071 - val_loss: 41.5254 - 215ms/epoch - 13ms/step
Epoch 9/50
16/16 - 0s - loss: 46.9615 - val_loss: 41.4718 - 231ms/epoch - 14ms/step
Epoch 10/50
16/16 - 0s - loss: 46.8676 - val_loss: 41.4456 - 236ms/epoch - 15ms/step
Epoch 11/50
16/16 - 0

Epoch 46/50
16/16 - 0s - loss: 40.7183 - val_loss: 50.6995 - 237ms/epoch - 15ms/step
Epoch 47/50
16/16 - 0s - loss: 40.7539 - val_loss: 50.6753 - 233ms/epoch - 15ms/step
Epoch 48/50
16/16 - 0s - loss: 40.6944 - val_loss: 50.7403 - 244ms/epoch - 15ms/step
Epoch 49/50
16/16 - 0s - loss: 40.7951 - val_loss: 50.7087 - 227ms/epoch - 14ms/step
Epoch 50/50
16/16 - 0s - loss: 40.7497 - val_loss: 50.8122 - 240ms/epoch - 15ms/step
10/10 [==============================] - 0s 5ms/step
Epoch 1/50
16/16 - 0s - loss: 42.3462 - val_loss: 52.2292 - 335ms/epoch - 21ms/step
Epoch 2/50
16/16 - 0s - loss: 41.6351 - val_loss: 50.5527 - 231ms/epoch - 14ms/step
Epoch 3/50
16/16 - 0s - loss: 41.3155 - val_loss: 51.0269 - 228ms/epoch - 14ms/step
Epoch 4/50
16/16 - 0s - loss: 41.1325 - val_loss: 51.3116 - 228ms/epoch - 14ms/step
Epoch 5/50
16/16 - 0s - loss: 41.0877 - val_loss: 51.2284 - 222ms/epoch - 14ms/step
Epoch 6/50
16/16 - 0s - loss: 41.0184 - val_loss: 51.2754 - 244ms/epoch - 15ms/step
Epoch 7/50
16/16 -

Epoch 42/50
16/16 - 0s - loss: 42.9126 - val_loss: 42.6111 - 221ms/epoch - 14ms/step
Epoch 43/50
16/16 - 0s - loss: 43.0114 - val_loss: 42.6960 - 244ms/epoch - 15ms/step
Epoch 44/50
16/16 - 0s - loss: 42.8602 - val_loss: 42.6226 - 259ms/epoch - 16ms/step
Epoch 45/50
16/16 - 0s - loss: 42.9221 - val_loss: 42.6439 - 238ms/epoch - 15ms/step
Epoch 46/50
16/16 - 0s - loss: 42.8819 - val_loss: 42.7173 - 239ms/epoch - 15ms/step
Epoch 47/50
16/16 - 0s - loss: 42.9475 - val_loss: 42.7453 - 234ms/epoch - 15ms/step
Epoch 48/50
16/16 - 0s - loss: 42.8742 - val_loss: 42.7053 - 241ms/epoch - 15ms/step
Epoch 49/50
16/16 - 0s - loss: 42.8247 - val_loss: 42.7325 - 226ms/epoch - 14ms/step
Epoch 50/50
16/16 - 0s - loss: 42.8864 - val_loss: 42.8493 - 240ms/epoch - 15ms/step
10/10 [==============================] - 0s 5ms/step
Epoch 1/50
16/16 - 0s - loss: 42.4271 - val_loss: 42.5478 - 341ms/epoch - 21ms/step
Epoch 2/50
16/16 - 0s - loss: 42.0011 - val_loss: 42.5482 - 247ms/epoch - 15ms/step
Epoch 3/50
16/

Epoch 38/50
16/16 - 0s - loss: 43.3336 - val_loss: 47.3249 - 247ms/epoch - 15ms/step
Epoch 39/50
16/16 - 0s - loss: 43.2975 - val_loss: 47.2128 - 234ms/epoch - 15ms/step
Epoch 40/50
16/16 - 0s - loss: 43.2834 - val_loss: 47.2909 - 215ms/epoch - 13ms/step
Epoch 41/50
16/16 - 0s - loss: 43.3158 - val_loss: 47.1301 - 251ms/epoch - 16ms/step
Epoch 42/50
16/16 - 0s - loss: 43.3585 - val_loss: 47.3529 - 225ms/epoch - 14ms/step
Epoch 43/50
16/16 - 0s - loss: 43.4080 - val_loss: 46.9939 - 209ms/epoch - 13ms/step
Epoch 44/50
16/16 - 0s - loss: 43.3684 - val_loss: 47.5751 - 229ms/epoch - 14ms/step
Epoch 45/50
16/16 - 0s - loss: 43.2871 - val_loss: 47.2285 - 202ms/epoch - 13ms/step
Epoch 46/50
16/16 - 0s - loss: 43.2777 - val_loss: 47.2942 - 212ms/epoch - 13ms/step
Epoch 47/50
16/16 - 0s - loss: 43.2302 - val_loss: 47.2124 - 227ms/epoch - 14ms/step
Epoch 48/50
16/16 - 0s - loss: 43.4010 - val_loss: 47.5733 - 230ms/epoch - 14ms/step
Epoch 49/50
16/16 - 0s - loss: 43.2273 - val_loss: 47.1193 - 244m

In [26]:
mse_arr_minmax = np.array(mse_50_Minmax)
mean_mse_minmax = np.mean(mse_arr_minmax)
std_mse_minmax = np.std(mse_arr_minmax)

In [27]:
print('The mean MSE after training the model 50 times after the data is scaled using minmax scaler is: ',mean_mse_minmax,
      ' and the standard deviation is: ', std_mse_minmax)

The mean MSE after training the model 50 times after the data is scaled using minmax scaler is:  74.74712459720027  and the standard deviation is:  45.93906271045597


This performance is better than the standard scaling that was performed before. part A the mean MSE and standard deviation was 58.269382589504986 and 18.803301390418138 respectively which is still better than both the standard scaler and minmax scaler. 

## using the Robust  scaler to scale the data instead of subtracting the mean and dividing by the standard deviation

In [28]:
# Splitting the raw data into training and testing and using the Robust scaler instance to fit and transform the training data
#and using the same transform on the test data:
#Scale features using statistics that are robust to outliers.
#This Scaler removes the median and scales the data according to
#the quantile range (defaults to IQR: Interquartile Range).
#The IQR is the range between the 1st quartile (25th quantile)
#and the 3rd quartile (75th quantile).

mse_50_robust = []
robust = RobustScaler()
model = regression_model()
for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.30)
    X_train_scaled = robust.fit_transform(X_train)
    X_test_scaled = robust.transform(X_test)
    model.fit(X_train_scaled, y_train, validation_split=0.3, epochs=50, verbose=2)
    predictions = model.predict(X_test_scaled)
    mse_50_robust.append(mean_squared_error(y_test, predictions))






Epoch 1/50
16/16 - 1s - loss: 1563.8579 - val_loss: 1566.5442 - 1s/epoch - 77ms/step
Epoch 2/50
16/16 - 0s - loss: 1549.2405 - val_loss: 1551.7395 - 270ms/epoch - 17ms/step
Epoch 3/50
16/16 - 0s - loss: 1535.1261 - val_loss: 1537.0065 - 287ms/epoch - 18ms/step
Epoch 4/50
16/16 - 0s - loss: 1520.9261 - val_loss: 1522.7347 - 238ms/epoch - 15ms/step
Epoch 5/50
16/16 - 0s - loss: 1506.9801 - val_loss: 1508.5178 - 244ms/epoch - 15ms/step
Epoch 6/50
16/16 - 0s - loss: 1493.0421 - val_loss: 1494.2106 - 247ms/epoch - 15ms/step
Epoch 7/50
16/16 - 0s - loss: 1479.0846 - val_loss: 1479.8419 - 233ms/epoch - 15ms/step
Epoch 8/50
16/16 - 0s - loss: 1465.0063 - val_loss: 1465.0977 - 241ms/epoch - 15ms/step
Epoch 9/50
16/16 - 0s - loss: 1450.4186 - val_loss: 1450.1176 - 242ms/epoch - 15ms/step
Epoch 10/50
16/16 - 0s - loss: 1435.5558 - val_loss: 1434.6239 - 229ms/epoch - 14ms/step
Epoch 11/50
16/16 - 0s - loss: 1420.2705 - val_loss: 1418.3956 - 221ms/epoch - 14ms/step
Epoch 12/50
16/16 - 0s - loss: 14

16/16 - 0s - loss: 157.7826 - val_loss: 119.0160 - 222ms/epoch - 14ms/step
Epoch 45/50
16/16 - 0s - loss: 155.5658 - val_loss: 118.1761 - 222ms/epoch - 14ms/step
Epoch 46/50
16/16 - 0s - loss: 153.6022 - val_loss: 117.4885 - 240ms/epoch - 15ms/step
Epoch 47/50
16/16 - 0s - loss: 151.6017 - val_loss: 116.7407 - 216ms/epoch - 13ms/step
Epoch 48/50
16/16 - 0s - loss: 149.7254 - val_loss: 116.2010 - 217ms/epoch - 14ms/step
Epoch 49/50
16/16 - 0s - loss: 147.9584 - val_loss: 115.6486 - 240ms/epoch - 15ms/step
Epoch 50/50
16/16 - 0s - loss: 146.2102 - val_loss: 115.1165 - 235ms/epoch - 15ms/step
10/10 [==============================] - 0s 4ms/step
Epoch 1/50
16/16 - 0s - loss: 142.5935 - val_loss: 150.6360 - 323ms/epoch - 20ms/step
Epoch 2/50
16/16 - 0s - loss: 141.1586 - val_loss: 149.6635 - 222ms/epoch - 14ms/step
Epoch 3/50
16/16 - 0s - loss: 139.8915 - val_loss: 148.7092 - 240ms/epoch - 15ms/step
Epoch 4/50
16/16 - 0s - loss: 138.4134 - val_loss: 147.8216 - 239ms/epoch - 15ms/step
Epoch 

Epoch 38/50
16/16 - 0s - loss: 90.0313 - val_loss: 79.8514 - 231ms/epoch - 14ms/step
Epoch 39/50
16/16 - 0s - loss: 89.5744 - val_loss: 79.4732 - 231ms/epoch - 14ms/step
Epoch 40/50
16/16 - 0s - loss: 89.0938 - val_loss: 78.9962 - 230ms/epoch - 14ms/step
Epoch 41/50
16/16 - 0s - loss: 88.6827 - val_loss: 78.4146 - 248ms/epoch - 15ms/step
Epoch 42/50
16/16 - 0s - loss: 88.1454 - val_loss: 77.9187 - 228ms/epoch - 14ms/step
Epoch 43/50
16/16 - 0s - loss: 87.6573 - val_loss: 77.4244 - 240ms/epoch - 15ms/step
Epoch 44/50
16/16 - 0s - loss: 87.1375 - val_loss: 76.8978 - 229ms/epoch - 14ms/step
Epoch 45/50
16/16 - 0s - loss: 86.6797 - val_loss: 76.3101 - 234ms/epoch - 15ms/step
Epoch 46/50
16/16 - 0s - loss: 86.1638 - val_loss: 75.8548 - 231ms/epoch - 14ms/step
Epoch 47/50
16/16 - 0s - loss: 85.6890 - val_loss: 75.2620 - 235ms/epoch - 15ms/step
Epoch 48/50
16/16 - 0s - loss: 85.2094 - val_loss: 74.8142 - 242ms/epoch - 15ms/step
Epoch 49/50
16/16 - 0s - loss: 84.7170 - val_loss: 74.1348 - 238m

Epoch 34/50
16/16 - 0s - loss: 55.1727 - val_loss: 50.1241 - 111ms/epoch - 7ms/step
Epoch 35/50
16/16 - 0s - loss: 54.9907 - val_loss: 50.0344 - 100ms/epoch - 6ms/step
Epoch 36/50
16/16 - 0s - loss: 54.7537 - val_loss: 49.7992 - 104ms/epoch - 6ms/step
Epoch 37/50
16/16 - 0s - loss: 54.5140 - val_loss: 49.5626 - 103ms/epoch - 6ms/step
Epoch 38/50
16/16 - 0s - loss: 54.3759 - val_loss: 49.3584 - 113ms/epoch - 7ms/step
Epoch 39/50
16/16 - 0s - loss: 54.1759 - val_loss: 49.3048 - 112ms/epoch - 7ms/step
Epoch 40/50
16/16 - 0s - loss: 54.0015 - val_loss: 49.1678 - 121ms/epoch - 8ms/step
Epoch 41/50
16/16 - 0s - loss: 53.8641 - val_loss: 49.0638 - 98ms/epoch - 6ms/step
Epoch 42/50
16/16 - 0s - loss: 53.6425 - val_loss: 48.8246 - 100ms/epoch - 6ms/step
Epoch 43/50
16/16 - 0s - loss: 53.4934 - val_loss: 48.8024 - 97ms/epoch - 6ms/step
Epoch 44/50
16/16 - 0s - loss: 53.3514 - val_loss: 48.5494 - 95ms/epoch - 6ms/step
Epoch 45/50
16/16 - 0s - loss: 53.1736 - val_loss: 48.3928 - 99ms/epoch - 6ms/s

Epoch 31/50
16/16 - 0s - loss: 43.5251 - val_loss: 45.1088 - 103ms/epoch - 6ms/step
Epoch 32/50
16/16 - 0s - loss: 43.4394 - val_loss: 45.1393 - 112ms/epoch - 7ms/step
Epoch 33/50
16/16 - 0s - loss: 43.4538 - val_loss: 45.1676 - 104ms/epoch - 6ms/step
Epoch 34/50
16/16 - 0s - loss: 43.3224 - val_loss: 45.0864 - 118ms/epoch - 7ms/step
Epoch 35/50
16/16 - 0s - loss: 43.3149 - val_loss: 45.0599 - 133ms/epoch - 8ms/step
Epoch 36/50
16/16 - 0s - loss: 43.2829 - val_loss: 45.0273 - 117ms/epoch - 7ms/step
Epoch 37/50
16/16 - 0s - loss: 43.2412 - val_loss: 44.9628 - 117ms/epoch - 7ms/step
Epoch 38/50
16/16 - 0s - loss: 43.1890 - val_loss: 44.9611 - 108ms/epoch - 7ms/step
Epoch 39/50
16/16 - 0s - loss: 43.1888 - val_loss: 44.8361 - 104ms/epoch - 6ms/step
Epoch 40/50
16/16 - 0s - loss: 43.1471 - val_loss: 44.8450 - 109ms/epoch - 7ms/step
Epoch 41/50
16/16 - 0s - loss: 43.0986 - val_loss: 44.8010 - 108ms/epoch - 7ms/step
Epoch 42/50
16/16 - 0s - loss: 43.1250 - val_loss: 44.9070 - 111ms/epoch - 7

...

Epoch 30/50
16/16 - 0s - loss: 35.1788 - val_loss: 32.0948 - 114ms/epoch - 7ms/step
Epoch 31/50
16/16 - 0s - loss: 35.1214 - val_loss: 32.1388 - 121ms/epoch - 8ms/step
Epoch 32/50
16/16 - 0s - loss: 35.0994 - val_loss: 32.1169 - 119ms/epoch - 7ms/step
Epoch 33/50
16/16 - 0s - loss: 35.0965 - val_loss: 32.1620 - 110ms/epoch - 7ms/step
Epoch 34/50
16/16 - 0s - loss: 35.0422 - val_loss: 32.1185 - 109ms/epoch - 7ms/step
Epoch 35/50
16/16 - 0s - loss: 35.0159 - val_loss: 32.1073 - 112ms/epoch - 7ms/step
Epoch 36/50
16/16 - 0s - loss: 34.9806 - val_loss: 32.1241 - 108ms/epoch - 7ms/step
Epoch 37/50
16/16 - 0s - loss: 34.9976 - val_loss: 32.1348 - 128ms/epoch - 8ms/step
Epoch 38/50
16/16 - 0s - loss: 34.9167 - val_loss: 32.1552 - 115ms/epoch - 7ms/step
Epoch 39/50
16/16 - 0s - loss: 34.9155 - val_loss: 32.1833 - 157ms/epoch - 10ms/step
Epoch 40/50
16/16 - 0s - loss: 34.9194 - val_loss: 32.1640 - 125ms/epoch - 8ms/step
Epoch 41/50
16/16 - 0s - loss: 34.8598 - val_loss: 32.1549 - 112ms/epoch - 

Epoch 27/50
16/16 - 0s - loss: 33.6756 - val_loss: 33.7637 - 114ms/epoch - 7ms/step
Epoch 28/50
16/16 - 0s - loss: 33.5483 - val_loss: 33.6567 - 132ms/epoch - 8ms/step
Epoch 29/50
16/16 - 0s - loss: 33.5369 - val_loss: 33.6108 - 134ms/epoch - 8ms/step
Epoch 30/50
16/16 - 0s - loss: 33.5623 - val_loss: 33.7391 - 123ms/epoch - 8ms/step
Epoch 31/50
16/16 - 0s - loss: 33.5871 - val_loss: 33.7073 - 109ms/epoch - 7ms/step
Epoch 32/50
16/16 - 0s - loss: 33.6233 - val_loss: 33.6871 - 125ms/epoch - 8ms/step
Epoch 33/50
16/16 - 0s - loss: 33.4978 - val_loss: 33.7819 - 105ms/epoch - 7ms/step
Epoch 34/50
16/16 - 0s - loss: 33.5578 - val_loss: 33.9578 - 114ms/epoch - 7ms/step
Epoch 35/50
16/16 - 0s - loss: 33.4728 - val_loss: 33.7889 - 125ms/epoch - 8ms/step
Epoch 36/50
16/16 - 0s - loss: 33.4674 - val_loss: 33.8364 - 105ms/epoch - 7ms/step
Epoch 37/50
16/16 - 0s - loss: 33.5755 - val_loss: 33.9163 - 131ms/epoch - 8ms/step
Epoch 38/50
16/16 - 0s - loss: 33.5003 - val_loss: 34.0549 - 116ms/epoch - 7

Epoch 24/50
16/16 - 0s - loss: 34.3242 - val_loss: 35.4187 - 116ms/epoch - 7ms/step
Epoch 25/50
16/16 - 0s - loss: 34.3378 - val_loss: 35.3167 - 111ms/epoch - 7ms/step
Epoch 26/50
16/16 - 0s - loss: 34.2914 - val_loss: 35.2374 - 128ms/epoch - 8ms/step
Epoch 27/50
16/16 - 0s - loss: 34.2778 - val_loss: 35.2304 - 115ms/epoch - 7ms/step
Epoch 28/50
16/16 - 0s - loss: 34.2731 - val_loss: 35.3494 - 116ms/epoch - 7ms/step
Epoch 29/50
16/16 - 0s - loss: 34.2302 - val_loss: 35.2972 - 120ms/epoch - 8ms/step
Epoch 30/50
16/16 - 0s - loss: 34.1572 - val_loss: 35.2752 - 120ms/epoch - 8ms/step
Epoch 31/50
16/16 - 0s - loss: 34.1253 - val_loss: 35.2448 - 111ms/epoch - 7ms/step
Epoch 32/50
16/16 - 0s - loss: 34.1256 - val_loss: 35.1546 - 104ms/epoch - 6ms/step
Epoch 33/50
16/16 - 0s - loss: 34.0838 - val_loss: 35.2571 - 111ms/epoch - 7ms/step
Epoch 34/50
16/16 - 0s - loss: 34.0927 - val_loss: 35.1788 - 112ms/epoch - 7ms/step
Epoch 35/50
16/16 - 0s - loss: 34.0527 - val_loss: 35.1860 - 112ms/epoch - 7

Epoch 21/50
16/16 - 0s - loss: 32.6492 - val_loss: 29.7204 - 110ms/epoch - 7ms/step
Epoch 22/50
16/16 - 0s - loss: 32.6184 - val_loss: 29.7931 - 106ms/epoch - 7ms/step
Epoch 23/50
16/16 - 0s - loss: 32.5748 - val_loss: 29.7205 - 108ms/epoch - 7ms/step
Epoch 24/50
16/16 - 0s - loss: 32.4990 - val_loss: 29.7398 - 135ms/epoch - 8ms/step
Epoch 25/50
16/16 - 0s - loss: 32.4794 - val_loss: 29.8039 - 133ms/epoch - 8ms/step
Epoch 26/50
16/16 - 0s - loss: 32.4703 - val_loss: 29.8388 - 101ms/epoch - 6ms/step
Epoch 27/50
16/16 - 0s - loss: 32.4916 - val_loss: 29.8073 - 109ms/epoch - 7ms/step
Epoch 28/50
16/16 - 0s - loss: 32.4645 - val_loss: 29.7839 - 113ms/epoch - 7ms/step
Epoch 29/50
16/16 - 0s - loss: 32.3689 - val_loss: 29.8706 - 103ms/epoch - 6ms/step
Epoch 30/50
16/16 - 0s - loss: 32.3097 - val_loss: 29.9028 - 120ms/epoch - 7ms/step
Epoch 31/50
16/16 - 0s - loss: 32.2775 - val_loss: 29.8874 - 114ms/epoch - 7ms/step
Epoch 32/50
16/16 - 0s - loss: 32.2688 - val_loss: 29.8564 - 112ms/epoch - 7

Epoch 18/50
16/16 - 0s - loss: 30.9628 - val_loss: 34.0794 - 109ms/epoch - 7ms/step
Epoch 19/50
16/16 - 0s - loss: 30.9501 - val_loss: 34.0219 - 110ms/epoch - 7ms/step
Epoch 20/50
16/16 - 0s - loss: 30.8754 - val_loss: 34.2055 - 128ms/epoch - 8ms/step
Epoch 21/50
16/16 - 0s - loss: 30.8115 - val_loss: 34.2944 - 108ms/epoch - 7ms/step
Epoch 22/50
16/16 - 0s - loss: 30.7960 - val_loss: 34.3002 - 119ms/epoch - 7ms/step
Epoch 23/50
16/16 - 0s - loss: 30.7785 - val_loss: 34.3282 - 110ms/epoch - 7ms/step
Epoch 24/50
16/16 - 0s - loss: 30.7401 - val_loss: 34.2820 - 108ms/epoch - 7ms/step
Epoch 25/50
16/16 - 0s - loss: 30.6283 - val_loss: 34.3630 - 106ms/epoch - 7ms/step
Epoch 26/50
16/16 - 0s - loss: 30.5886 - val_loss: 34.3366 - 106ms/epoch - 7ms/step
Epoch 27/50
16/16 - 0s - loss: 30.6355 - val_loss: 34.2309 - 110ms/epoch - 7ms/step
Epoch 28/50
16/16 - 0s - loss: 30.5062 - val_loss: 34.3768 - 103ms/epoch - 6ms/step
Epoch 29/50
16/16 - 0s - loss: 30.4972 - val_loss: 34.4116 - 127ms/epoch - 8

In [33]:
mse_arr_robust = np.array(mse_50_robust)
mean_mse_robust = np.mean(mse_arr_robust)
std_mse_robust = np.std(mse_arr_robust)

In [34]:
print('The mean MSE after training the model 50 times after the data is scaled using robust scaler is: ',mean_mse_robust,
      ' and the standard deviation is: ', std_mse_robust)

The mean MSE after training the model 50 times after the data is scaled using robust scaler is:  53.39723753448831  and the standard deviation is:  77.20912119999225


This performance than part A where mean MSE and standard deviation was 58.269382589504986.
However, it haas more standard deviation than part A. The standard deviation in part A was 18.803301390418138 compared to the cuurent standard deviation of 77.20912119999225